<a href="https://colab.research.google.com/github/zakirovde/netology_dz/blob/master/HW2_banki_sent_vocab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 7.8MB/s 


In [0]:
import json
import bz2
import regex
from tqdm.notebook import tqdm
tqdm.pandas()
from scipy import sparse
import pandas as pd
import numpy as np
import nltk
import re
from nltk import FreqDist
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import toktok
toktok = toktok.ToktokTokenizer()
%matplotlib inline
%pylab inline
from sklearn.metrics import precision_score, recall_score, \
  f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from pymorphy2 import MorphAnalyzer
pm2 = MorphAnalyzer()
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from gensim.models import  *
from gensim import similarities
from gensim.corpora import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import *
from gensim.corpora import Dictionary
from gensim.models import *
from gensim import similarities
import warnings
warnings.filterwarnings('ignore')
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Populating the interactive namespace from numpy and matplotlib


In [0]:
df = pd.read_csv('/content/drive/My Drive/banki_responses.csv')
df.text_prep = df.text_prep.fillna('')

In [0]:
sentences = []

for i in range(df.shape[0]):
  if len(df.text_prep[i])>0:
    sentences += [df.text_prep[i].split()]

In [0]:
model = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

In [0]:
print(model.wv.most_similar('женщина', topn=4))

[('дама', 0.7781516313552856), ('мужчина', 0.7652180194854736), ('женщины', 0.7406284809112549), ('мадам', 0.7291651964187622)]


In [0]:
print(model.wv.most_similar(positive=["женщина", "сотрудник"], negative=["мужчина"], topn=1))
print(model.wv.most_similar(positive=["работники", "женщина"], negative=["работник"], topn=1))
print([i[0] for i in model.wv.most_similar("втб", topn=10)])
print(model.wv.doesnt_match("втб отп сбербанк мужчина".split()))

[('сотрудница', 0.7255154252052307)]
[('женщины', 0.6390053033828735)]
['гута', 'бм', 'ткб', 'райффайзенбанк', 'транскредитбанк', 'уралсиб', 'юникредитбанк', 'юникредит', 'транскредит', 'транскредитбанка']
мужчина


А что? Выглядит довольно правдоподобно!

In [0]:
fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

In [0]:
top_words = []

for w in fd.most_common(200):
    top_words.append(w[0])

top_words_vec = model[top_words]

In [0]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 4.1 s, sys: 15.2 ms, total: 4.12 s
Wall time: 1.09 s


In [0]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (model, top 200 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

На графике видно, что можно выделить в группы такие слова, как, например:

1. Погашение, погасить, оплата, оплатить (оплата);
2. Средства, деньги, деньга (деньги);
3. Оператор, сотрудник, менеджер, специалист (сотрудники);
4. Год, месяц, неделя (время);
5. Предложить, попросить, просьба, просить (просьба).



In [0]:
bank_list = ['гута',  'бм', 'ткб', 'райффайзенбанк', 'транскредитбанк',
 'уралсиб', 'юникредитбанк', 'юникредит', 'транскредит',
 'возрождение', 'газпромбанк', 'псб', 'райффайзен', 'мдм',
 'ммб', 'промсвязьбанк', 'абсолют', 'газпром', 'росбанк',
 'росевробанк', 'внешторгбанк', 'убрир', 'телебанк',
 'татфондбанк', 'интеза', 'райф', 'импэксбанк', 'россельхозбанк', 'сбер',
 'сбербанк', 'союз', 'сити', 'райфайзенбанк', 'втб', 'ткс', 'тинькофф']

bank_list_vec = model[bank_list]

In [0]:
fast = model.wv['быстрый'] - model.wv['медленный']
good = model.wv['хороший'] - model.wv['плохой']

x, y = [], []
for i in bank_list:
  x.append(np.dot(model.wv[i], good.T))
  y.append(np.dot(model.wv[i], fast.T))

In [0]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="2 fast 2 good")

source = ColumnDataSource(data=dict(x1=x,
                                    x2=y,
                                    names=bank_list))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.xaxis.axis_label = 'Good'
p.yaxis.axis_label = 'Fast'

show(p)